In [ ]:
from pyspark import pipelines as dp
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
@dp.table(name='fact_stock_prices_gold')
def fact_stock_prices_gold():
    df = spark.read.table('stock_prices.stocks_dlt.stock_prices_silver')
    df_agg = (
        df.filter(col('trade_date').isNotNull())
            .withColumn('trade_year', year('trade_date'))
            .groupBy('ticker', 'trade_year')
            .agg(
                round(avg('close_price'), 2).alias('avg_close_price'),
                round(avg('volume'), 2).alias('avg_volume')
            )
    )

    return df_agg

In [ ]:
@dp.table(name='dim_stock')
def dim_stock():
    df = spark.read.table('stock_prices.stocks_dlt.stock_prices_silver')

    dim_df = (
        df.select(
            'ticker',
            'brand_name',
            'industry_tag',
            'country'
        )
        .dropDuplicates(['ticker'])
    )

    return dim_df